## Programa general

En preparacion para hacer un despliegue en dash para hacer mas dinamico el analalis

In [ ]:
class CargadorCSV:
    """Clase para carga de archivos con formato .csv"""
    def __init__(self, file_path):
        """
        Constructor de la clase. Recibe la ruta del archivo.
        """
        self.file_path = file_path
    
    def visualizar_avance(self, bytes_avance, bytes_completo):
        """Visualiza el avance de la carga del archivo en porcentaje."""
        avance = min(round(bytes_avance / bytes_completo, 2) * 100, 100)
        clear_output(wait=True)
        print(f"Avance estimado: {avance}%")
    
    def crear_csv_por_horarios(self, chunk_size=100000, sep="|"):
        """Crea archivos CSV separados por horarios a partir del archivo original."""
        try:
            bytes_completo = os.path.getsize(self.file_path)
            encabezado = True
            horarios = ['MEDIODIA', 'TARDE', 'NOCHE', 'MANANA']
            
            for chunk in pd.read_csv(self.file_path, sep=sep, chunksize=chunk_size):
                bytes_avance = 0
                
                if encabezado:
                    for clasificador_horario in horarios:
                        df_filtered = chunk.loc[chunk["HORARIO"] == clasificador_horario].drop("HORARIO", axis=1)
                        file_name = f"{self.file_path[:-4]}_{clasificador_horario}.csv"
                        df_filtered.to_csv(file_name, index=False)
                        bytes_avance += os.path.getsize(file_name)
                    df_Nofiltered = chunk.loc[~chunk["HORARIO"].isin(horarios)]
                    df_Nofiltered.to_csv(f"{self.file_path[:-4]}_ValoresPerdidos.csv ", index=False)
                    
            #df_MANANA_filtered = df_MANANA.loc[~df_MANANA["MES"].isin(months_to_exclude)]
                    encabezado = False
                else:
                    for clasificador_horario in horarios:
                        df_filtered = chunk.loc[chunk["HORARIO"] == clasificador_horario].drop("HORARIO", axis=1)
                        file_name = f"{self.file_path[:-4]}_{clasificador_horario}.csv"
                        df_filtered.to_csv(file_name, mode='a', index=False, header=False)
                        bytes_avance += os.path.getsize(file_name)
                    df_Nofiltered = chunk.loc[~chunk["HORARIO"].isin(horarios)]
                    df_Nofiltered.to_csv(f"{self.file_path[:-4]}_ValoresPerdidos.csv ",mode = 'a', index=False, header=False)                   
                
                self.visualizar_avance(bytes_avance, bytes_completo)
            #clear_output(wait=True)
            print(f"Proceso terminado")
        except Exception as error:
            print("Ocurrió un error:", error)

In [1]:
#Resumen

# Librerias a utilizar, por defecto ya vienen instaladas en el entorno "pip install git jupyterlab numpy pandas plotly "
# para mejor visualizacion de los df y manejo de estadisticas:
# ! pip install pandasgui
# ! pip install pandas-profiling
# ! pip install seaborn
# ! pip install scipy
#! pip install optbinning
#! pip install ortools == 9.4.1874
#! pip install -U scikit-learn
import os
import datetime
from IPython.display import clear_output
import pandas as pd
import numpy as np
from pandasgui import show
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats import kurtosis
import statsmodels.api as sm
from optbinning import OptimalBinning
from optbinning import BinningProcess
import sklearn
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
print("librerias cargadas correctamente") 

class revisor_data_csv:
    """Clase para carga y revisión de archivos con formato .csv
    Con esto podemos visualizar y manipular la data de forma controlada minimizando errores y
    pudiendo posteriormente crar un pipeline del proceso ETL"""
    def __init__(self, file_path):
        """
        Constructor de la clase que recibe la ruta del archivo y carga el archivo
        Se asume archivos .csv estándar, sep = ","
        """
        self.file_path = file_path
        try:
            # Se carga el df 
            self.mensaje("cargando", self.file_path)
            self.df = pd.read_csv(self.file_path)
            self.mensaje("cargado")
        except Exception as error:
            print("Ocurrió un error:", error)
            
    def mensaje(self, tipo, arg=None):
        tipos_mensajes = {"cargando": f"Cargando archivo {arg}",
                          "cargado": "Carga completa"}
        if tipo in tipos_mensajes:
            clear_output(wait=True)
            print(tipos_mensajes[tipo])
        else:
            clear_output(wait=True)
            print("Tipo de mensaje no válido")
            print(f"{list(tipos_mensajes.keys())}")
    
    def analisis_nulos(self):
        cuenta_nulos = self.df.isnull().sum()
        cuenta_nulos = cuenta_nulos[cuenta_nulos != 0]
        porcentaje_nulos = round((cuenta_nulos / self.df.shape[0]) * 100, 2)
        return pd.DataFrame({"cant_nulos": cuenta_nulos, "porcentaje_nulos": porcentaje_nulos})

    def nulos_EliminacionSimple(self, lista):
        """Se eliminarán las filas que contengan valores nulos en las columnas indicadas por la lista"""
        try:
            self.df = self.df.dropna(subset=lista)
        except Exception as error:
            print("Ocurrió un error:", error)

    def nulos_MediaCondicional(self, variable, lista):
        """Se asignará a los valores nulos de la columna variable,
        el valor promedio agrupando con las columnas indicadas por la lista"""
        try:
            self.df.loc[self.df[variable].isnull(), variable] = self.df.groupby(lista, as_index=False)[[variable]].transform('mean')
        except Exception as error:
            print("Ocurrió un error:", error)

    def eliminar_por_condicion(self,columna,condicion):
        """Se eliminarán las filas que contengan valores 'condicion' en la columna indicadas por la lista"""
        try:
            self.df = self.df.drop(self.df[self.df[columna] == condicion].index)
        except Exception as error:
            print("Ocurrió un error:", error)

#revision = revisor_data_csv("..\\datos_internos/20210513_Challenge_AA_MANANA.csv")
revision = revisor_data_csv("df_prueba_MANANA.csv")

# Eliminacion de valores nulos
revision.nulos_EliminacionSimple(['MORAS', 'SEXO', 'ESTADOCIVIL', 'FECHANACIMIENTO', 'ANTIGUEDAD', 'EDAD'])
revision.nulos_MediaCondicional('INGRESO',['ESTADO','EDAD','ESTADOCIVIL','SEXO'])
revision.nulos_EliminacionSimple(['INGRESO'])
revision.df['MORAS'] = revision.df['MORAS'].astype(int)
revision.df['ANTIGUEDAD'] = revision.df['ANTIGUEDAD'].astype(int)
revision.df['EDAD'] = revision.df['EDAD'].astype(int)
revision.df['FECHANACIMIENTO'] = pd.to_datetime(revision.df['FECHANACIMIENTO']).dt.date
revision.df['FECHANACIMIENTO'] = pd.to_datetime(revision.df['FECHANACIMIENTO'])
revision.eliminar_por_condicion('ESTADOCIVIL',' ')

revision.df = revision.df.drop(['CLIENTE'], axis=1)

class preprocesamiento:
    def __init__(self,revisor_data):
        """
        Constructor de la clase recibe el objeto instanciado de la clase revisor_data_csv, 
        seria mejor usar un patron de diseño DataFrameSingleton pero esta division solo es 
        para fines demostrativos,en codigo final todos los metodos deben 
        pertenecer a una sola clase.
        """
        try:
            # Se carga el df 
            self.revisor_data = revisor_data
            self.df = self.revisor_data.df
            self.matriz_correlacion = None
            self.transformaciones = {}
        except Exception as error:
            print("Ocurrió un error:", error)

    #Este metodo esta repetido en la clase graficos, al final se debe mejorar el codigo ya sea uniendo todas las clases o usando herencias.
    def varibles_numericas(self):
        """Devuelve una lista con las variables numericas de un df"""
        return [column_name for column_name, data_type in zip(self.df.columns, self.df.dtypes) if ((data_type != 'category') and  np.issubdtype(data_type, np.number))]
    
    def detecion_outlier(self,nombre_columna,q=.1):
        """ 
        Funcion para detectar valores atípicos (utiliers) de una columna especifica usando cuantiles.
        Por defecto se usa Deciles / dividiendo la distribucion en 10 partes, pero ajsutando el valor de
          q = 0.25 se trabajaria con cuartiles.
        La funcion devuelve una "lista/pandas.core.indexes.numeric.Int64Index"  con los indices de los 
        valores atípicos del df
        """
        # try:
        #calculo de cuantiles
        Q1 = self.df[nombre_columna].quantile(q)
        Q3 = self.df[nombre_columna].quantile(1-q)
        IQR = Q3-Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        indice_filas_eliminar = self.df.index[(self.df[nombre_columna] < limite_inferior) | (self.df[nombre_columna] > limite_superior) ]
        return indice_filas_eliminar   

    def eliminacion_outlier(self,nombre_columna,q=0.1):
        """ 
        Funcion para eliminar valores atípicos (outliers) de una columna especifica usando cuantiles.
        La funcion no devuelve nada porque los cambios se hacen en el df que se pasa por referencia
        """
        try:
            if nombre_columna in self.varibles_numericas():
                indices = self.detecion_outlier(nombre_columna,q)
                self.df = self.df.drop(indices)
                self.df.reset_index(inplace=True,drop=True)
            else:
                print("La variable no es numerica")
        except Exception as error:
            print("Ocurrió un error:", error)
            
    def guardar_transformador(self,nombre_columna,transformador):
        """Metodo para guardar transformador aplicado, se almacenan en lista, si se
        aplican dos o mas se almacenaran en la secuencia aplicada"""
        if nombre_columna in list(self.transformaciones.keys()):
            self.transformaciones[nombre_columna] = self.transformaciones[nombre_columna]+[transformador]
        else:
            self.transformaciones[nombre_columna] = [transformador]
            
    def Transf_MinMaxScaler(self,nombre_columna):
        """crea objeto que Transforma los valores de la columna indicada 
        usando MinMaxScaler de sklear, devuelve el objeto para transformar futuros valores"""
        try:
            if nombre_columna in self.varibles_numericas():
                scaler = preprocessing.MinMaxScaler()
                escalador = scaler.fit(self.df[nombre_columna].values.reshape(-1, 1))
                return escalador
            else:
                print("La variable no es numerica")
        except Exception as error:
            print("Ocurrió un error:", error)

    def Transf_Quantile(self,nombre_columna):
        """crea objeto para Transformar los valores de la columna indicada  
        usando  QuantileTransformer de sklearn"""
        try:
            if nombre_columna in self.varibles_numericas():
                scaler = preprocessing. QuantileTransformer()
                escalador = scaler.fit(self.df[nombre_columna].values.reshape(-1, 1))
                return escalador
            else:
                print("La variable no es numerica")
        except Exception as error:
            print("Ocurrió un error:", error)   

    def Transf_OneHot_binario(self, nombre_columna):
        """Crea objeto para transformacion  OneHot cuando la varible es binaria, devuelve
        el transformador ya entrenado"""
        try:
            value_var = self.df[nombre_columna].astype("category")
            codificador_oneHot = OneHotEncoder(handle_unknown='ignore', drop='first')
            codificacion = codificador_oneHot.fit(pd.DataFrame(value_var, columns=[nombre_columna]))
            return codificacion
        except Exception as error:
            print("Ocurrió un error:", error)

    def Transf_OneHot(self, nombre_columna):
        """crea objeto para Transformar los valores de la columna indicada  
        usando  OneHot encoder de sklearn, devuelve el transformador entrenado """
        try:
            value_var = self.df[nombre_columna].astype("category")
            codificador_oneHot = OneHotEncoder(handle_unknown='ignore')
            codificacion = codificador_oneHot.fit(pd.DataFrame(value_var, columns=[nombre_columna]))
            return codificacion
        except Exception as error:
            print("Ocurrió un error:", error)

    def Transf_woe(self,nombre_columna,nombre_target):
        """crea el objeto para Transformar los valores de la columna indicada  
        usando woe de OptimalBinning,
        Distinge de variables numericas y categoricas
        Devuelve el objeto transoformador"""
        try:
            data_type = self.df[nombre_columna].dtypes
            target = self.df[nombre_target]
            x = self.df.loc[:,nombre_columna]
            if (data_type == 'object' or data_type.name == 'category'):  
                optb = OptimalBinning(name = nombre_columna,dtype ='categorical',solver='mip')
                optb.fit(x,target)
            elif np.issubdtype(data_type, np.number):
                optb = OptimalBinning(name = nombre_columna,dtype = 'numerical',solver='cp')
                optb.fit(x,target)
            else:
                print("La variable se de convertir a tipo numerica o categorica")
                optb = None
                
            return optb
        except Exception as error:
            print("Ocurrió un error:", error)        

    def calc_matriz_correlacion(self, columnas=None,filas=None):
        """metodo para la creacion de la matriz de correlacion, se pude ajustar el numero de filas y las columnas
        a calcular para la correlacion"""
        try:
            if self.matriz_correlacion is None or self.matriz_correlacion.empty:
                if columnas:
                    # Se confirma que las columnas existan y sean numericas
                    columnas = [ col for col in columnas if col in self.varibles_numericas()]
                else:
                    columnas = self.varibles_numericas()
                if filas:
                    data_set = self.df[columnas].sample(filas)
                    data_set.reset_index(inplace=True,drop=True)
                else:
                    data_set = self.df[columnas]
                self.matriz_correlacion = data_set.corr(method='pearson', numeric_only=True)
        except Exception as error:
            print("Ocurrió un error:", error)    

    def clasificacion_correlacion(self, target_name ,nivel_correlacion_target = 0.3, nivel_correlacion_inter_variables = 0.3):
        """Metodo para clasificar las variables en funcion del nivel de correlacion 
        1. Entre las variables y el objetivo (target_name), escogiendo las que tengan una 
        Moderada correlación |corr| > 0.3 pero se puede ajustar si se desea.
        2. Entre las mismas variables permitiendo hasta una Moderada correlación |corr| < 0.3  entre ellas
        con el objetivo de reducir la multicolinalidad"""

        try:
            # Calculo de la correlacion para todas las variables
            self.calc_matriz_correlacion()
            matriz = self.matriz_correlacion

            #Verificacion de la correlacion entre todas las variables y el target, ordenadas de mayor a menor
            signals = []
            for i in range(len(matriz)):
                for j in range(i+1, len(matriz)):
                    signal_1 = matriz.columns[i]
                    signal_2 = matriz.columns[j]
                    correlation = abs(matriz.iloc[i, j])
                    signals.append([signal_1, signal_2, correlation])
            df_correlacion = pd.DataFrame(signals, columns=['sig_1', 'sig_2', 'nivel_correlacion'])
            df_correlacion_mod = df_correlacion[df_correlacion['sig_2'] == target_name].sort_values(by='nivel_correlacion', ascending=False)
            variables_x = list(df_correlacion_mod [df_correlacion_mod['nivel_correlacion']>nivel_correlacion_target]['sig_1'])

            # Verificacion de la correlacion entre variables 
            for i, variable1 in enumerate(variables_x):
                for variable2 in variables_x[i+1:]:
                    mask = (df_correlacion['sig_1'] == variable1) & (df_correlacion['sig_2'] == variable2)
                    val_corr = list(df_correlacion[mask]['nivel_correlacion'])
                    if not val_corr:
                        mask = (df_correlacion['sig_1'] == variable2) & (df_correlacion['sig_2'] == variable1)
                        val_corr = list(df_correlacion[mask]['nivel_correlacion'])
                    val_corr = val_corr[0] if val_corr else 0
                    if val_corr > nivel_correlacion_inter_variables:
                        if variable2 in variables_x:
                            variables_x.remove(variable2)
            return variables_x
        except Exception as error:
            print("Ocurrió un error:", error)

    
    def crea_EDAD_NORM(self,nombre_columna = 'EDAD'):
        """Para crear una variable EDAD normalizada se usa como referencia la fecha actual,
        independiente del momento en que se corra el programa, se calcula la diferencia entre
        la fecha de nacimiento provista y la fecha actual, se aplica una eliminacion de outliers
        y una transformacion estandar normalizando entre el valor minimo y maximo.
        """
        try:
            fecha_actual = datetime.datetime.now()
            self.df[nombre_columna] = (fecha_actual - self.df['FECHANACIMIENTO']).dt.days // 365
            self.eliminacion_outlier(nombre_columna)
            escalador = self.Transf_MinMaxScaler(nombre_columna) 
            self.df[nombre_columna] = escalador.transform(self.df[nombre_columna].values.reshape(-1, 1)) 
            self.guardar_transformador(nombre_columna,escalador)
            self.df.drop(columns=['ANIO', 'MES','FECHANACIMIENTO'], inplace=True)
        
            # Se actualiza la instancia de la clase revisor_data_csv
            #self.revisor_data.df = self.df
        except Exception as error:
            print("Ocurrió un error:", error)

    def mod_columna_Transf_Quantile(self,nombre_columna):
        """Modifica la columna indicada utilizando el Transf_Quantile, 
        alamcena el nombre de la columna modificada y el objeto utilizado para la transformacion"""
        try:
            escalador = self.Transf_Quantile(nombre_columna)
            self.df[nombre_columna] = escalador.transform(self.df[nombre_columna].values.reshape(-1, 1))
            self.guardar_transformador(nombre_columna,escalador)
            
            # Se actualiza la instancia de la clase revisor_data_csv
            #self.revisor_data.df = self.df
        except Exception as error:
            print("Ocurrió un error:", error)

    def mod_columna_OneHot_binario(self,nombre_columna):
        """Modifica la columna indicada utilizando OneHot_binario, 
        almacena el nombre de la columna modificada y 
        el objeto utilizado para la transformacion"""
        try:
            escalador = self.Transf_OneHot_binario(nombre_columna)
            arreglo = escalador.transform(preproceso.df[[nombre_columna]]).toarray()
            self.df[nombre_columna] = arreglo.astype(int)
            self.guardar_transformador(nombre_columna,escalador)
    
             # Se actualiza la instancia de la clase revisor_data_csv
            #self.revisor_data.df = self.df
        except Exception as error:
            print("Ocurrió un error:", error)

    def mod_columna_OneHot(self,nombre_columna):
        try:
            escalador = self.Transf_OneHot(nombre_columna)
            arreglo = escalador.transform(preproceso.df[[nombre_columna]]).toarray()
            columnas_codificadas = escalador.get_feature_names_out([nombre_columna])
            df_codificado = pd.DataFrame(arreglo, columns=columnas_codificadas)

            #Se introducen en df los valores codificados
            pos = self.df.columns.get_loc(nombre_columna)
            for col in df_codificado.columns:
                self.df.insert(pos, col, df_codificado[col])
                pos += 1
            self.df.drop(columns=[nombre_columna], inplace=True)
            self.guardar_transformador(nombre_columna,escalador)
            
             # Se actualiza la instancia de la clase revisor_data_csv
            #self.revisor_data.df = self.df
        except Exception as error:
            print("Ocurrió un error:", error)
        
    def mod_woe(self,nombre_columna,nombre_target,metrica = "woe"):
        """Metodo para transformar los elementos de una columna 
        usando woe de OptimalBinning, se pueden utilizar otras metricas 
        como: "event_rate", "woe", "indices" and "bins" ."""
        try:
            escalador = self.Transf_woe(nombre_columna,nombre_target)
            x = self.df.loc[:,nombre_columna]
            self.df[nombre_columna] = escalador.transform(x, metric=metrica)
            self.guardar_transformador(nombre_columna,escalador)

             # Se actualiza la instancia de la clase revisor_data_csv
            #self.revisor_data.df = self.df
        except Exception as error:
            print("Ocurrió un error:", error)  
preproceso = preprocesamiento(revision)

Carga completa


In [2]:
preproceso = preprocesamiento(revision)
preproceso.df = preproceso.df.sample(1000)
preproceso.crea_EDAD_NORM('EDAD')

X = preproceso.df.drop(columns=['TARGET'])
y = preproceso.df['TARGET'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=69)


pipes_bining = []
for columna in X_train.columns:
    pipes_bining.append((columna + '_Tr', Pipeline([('columna', BinningProcess(variable_names=[columna]))]), [columna]))
preprocessor = ColumnTransformer(pipes_bining)
#preprocessor.fit(X_train, y_train)
# X_train_transf = preprocessor.transform(X_train)

pipe = Pipeline((
    ("BinningProcess", preprocessor),
    ("Principal_component_analysis", PCA(n_components=5)),
    ("KNeighborsClassifier", KNeighborsClassifier(n_neighbors=5))))
# pipe.fit(X_train, y_train)
# prediccion= pipe.predict(X_test)


# para revisar los parametros que se pueden utilizar en Grid
# pipe.get_params()
modelo_final = GridSearchCV(estimator = pipe,
                     param_grid = {'KNeighborsClassifier__n_neighbors': list(range(6,15))},
                     cv = 3)
modelo_final.fit(X_train, y_train)           
resultados = pd.DataFrame(modelo_final.cv_results_)
resultados

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_KNeighborsClassifier__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,1.111830,0.103701,0.036024,1.414505e-03,6,{'KNeighborsClassifier__n_neighbors': 6},0.820513,0.776824,0.845494,0.814277,0.028379,9
1,1.115473,0.121704,0.036357,3.302009e-03,7,{'KNeighborsClassifier__n_neighbors': 7},0.816239,0.806867,0.871245,0.831450,0.028398,1
2,1.173507,0.125247,0.037025,1.633264e-03,8,{'KNeighborsClassifier__n_neighbors': 8},0.803419,0.798283,0.854077,0.818593,0.025179,8
3,1.109832,0.109358,0.035357,1.885987e-03,9,{'KNeighborsClassifier__n_neighbors': 9},0.824786,0.798283,0.862661,0.828577,0.026418,5
4,1.091061,0.098866,0.034023,4.052337e-07,10,{'KNeighborsClassifier__n_neighbors': 10},0.820513,0.789700,0.862661,0.824291,0.029906,7
5,1.091106,0.102526,0.034356,4.718200e-04,11,{'KNeighborsClassifier__n_neighbors': 11},0.824786,0.806867,0.862661,0.831438,0.023258,2
6,1.101401,0.105851,0.034356,4.712018e-04,12,{'KNeighborsClassifier__n_neighbors': 12},0.824786,0.806867,0.858369,0.830007,0.021347,3
7,1.093063,0.098161,0.037024,3.560934e-03,13,{'KNeighborsClassifier__n_neighbors': 13},0.824786,0.802575,0.854077,0.827146,0.021092,6
8,1.091395,0.099469,0.034690,4.716513e-04,14,{'KNeighborsClassifier__n_neighbors': 14},0.824786,0.806867,0.854077,0.828577,0.019459,4
